In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_image,train_label),(test_image,test_label)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
train_image=train_image/255
test_image=test_image/255


In [6]:
train_image[0].shape

(28, 28)

In [7]:
train_image=train_image.reshape(len(train_image),28,28,1)
test_image=test_image.reshape(len(test_image),28,28,1)

In [8]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                          hp.Choice('conv_1_kernel',values=[3,5]),
                          activation='relu',
                          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                          hp.Choice('conv_2_kernel',values=[3,5]),
                          activation='relu'
      
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16 ),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

  


In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


<ipython-input-9-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='output',project_name='fashion_mnist')

In [11]:
tuner_search.search(train_image,train_label,epochs=5,validation_split=0.1)

Trial 10 Complete [00h 00m 36s]
val_accuracy: 0.9169999957084656

Best val_accuracy So Far: 0.9169999957084656
Total elapsed time: 00h 08m 30s


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        38432     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,774,810
Trainable params: 1,774,810
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_image,train_label,epochs=30,validation_split=0.2,initial_epoch=5)

Epoch 6/30
1500/1500 [==============================] - 7s 4ms/step - loss: 0.0922 - accuracy: 0.9656 - val_loss: 0.2044 - val_accuracy: 0.9396
Epoch 7/30
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0613 - accuracy: 0.9782 - val_loss: 0.2309 - val_accuracy: 0.9383
Epoch 8/30
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0463 - accuracy: 0.9833 - val_loss: 0.2587 - val_accuracy: 0.9358
Epoch 9/30
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0336 - accuracy: 0.9881 - val_loss: 0.2834 - val_accuracy: 0.9298
Epoch 10/30
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0270 - accuracy: 0.9904 - val_loss: 0.3350 - val_accuracy: 0.9315
Epoch 11/30
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0229 - accuracy: 0.9919 - val_loss: 0.3869 - val_accuracy: 0.9293
Epoch 12/30
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0214 - accuracy: 0.9924 - val_loss: 0.3860 - val_accura

In [15]:
test_pred=model.predict(test_image)

313/313 [==============================] - 1s 2ms/step


In [16]:
test_label_pred=np.argmax(test_pred,axis=1)

In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,precision_score,recall_score
cm=confusion_matrix(test_label,test_label_pred)
cm


array([[796,   4,  14,  18,   1,   1, 162,   0,   4,   0],
       [  1, 987,   0,   7,   1,   0,   2,   0,   2,   0],
       [ 17,   3, 852,  12,  48,   0,  65,   0,   3,   0],
       [ 10,  19,  13, 897,  11,   0,  49,   0,   0,   1],
       [  1,   3,  58,  23, 837,   0,  77,   0,   1,   0],
       [  0,   0,   0,   0,   0, 981,   0,  11,   0,   8],
       [ 78,   5,  52,  21,  62,   0, 776,   0,   6,   0],
       [  0,   0,   0,   0,   0,   7,   0, 965,   1,  27],
       [ 10,   5,   4,   3,   3,   3,   8,   4, 960,   0],
       [  0,   0,   1,   0,   0,   2,   0,  31,   0, 966]])

In [21]:
accuracy=accuracy_score(test_label,test_label_pred)
accuracy

0.9017

In [23]:
f1score=f1_score(test_label,test_label_pred,average='weighted')
f1score

0.9024927165424684